# Data Preparation

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import random
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [2]:
#Read each line in train/test file and split word and its part-of-speech by '/' seperator
def read(file):
    with open(file, 'r', encoding='utf8') as f:
        file = f.read().splitlines()
    data = [[] for _ in range(len(file))]
    for idx, i in enumerate(file):
        a = i.split()
        for j in a:
            tmp = (j.rsplit('/',1))
            data[idx].append((tmp[0], tmp[1]))
    return data

In [3]:
#Load training corpus and testing file to memory
train_set = read('C:/Users/owcap/Documents/Learning/CS221/Project/corpus/train.txt')
test_set = read('C:/Users/owcap/Documents/Learning/CS221/Project/corpus/test.txt')

In [4]:
#Create list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

In [5]:
#Let's check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))

16


In [6]:
unique_tags = len(tags)

In [7]:
#Let's check how many words are present in vocabulary
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

1028


### POS Tagging using Hidden Markov Model (HMM)

We'll use the HMM algorithm to tag the words. Given a sequence of words to be tagged, the task is to assign the most probable tag to the word. 
In other words, to every **word w**, assign **the tag t** that maximises the likelihood **$P(t|w)$**. 

Since $$P(t|w) = {{P(w|t)*P(t)} \over {P(w)}}$$ 
After ignoring $P(w)$, we have to compute $P(w|t)$ and $P(t)$.

Now:
* **$P(w|t)$: is the emission probability** of a given word for a given tag. This can be computed based on the fraction of given word for given tag to the total count of that tag, ie: $$P(w|t) = count(w, t) \over count(t)$$. However, if the training data is small and not cover enough cases, the emission probability can lead to zero. Therefore we need to use a method called **Laplace smoothing** which adds a value to the numerator to prevent zero probability. The equation above can be written:
$$P(w|t) = count(w, t) + 1 \over count(t)+ N$$  which N is the number of unique tags in training data



* **$P(t)$: is the probability of tag (also transition probability)**, and in a tagging task, we assume that a tag will depend only on the previous tag (Markov order 1 assumption). In other words, the probability of say a tag being NOUN will depend only on the previous tag $t(n-1)$. So for e.g. if $t(n-1)$ is a ADJ, then $t(n)$ is likely to be an NOUN since adjectives often precede a noun (blue coat, tall building etc.).



# Build the Viterbi POS tagger

#### Function to compute emission probabilties for a given word

In [8]:
#Compute emission probability for a given word for a given tag
def word_given_tag(word, tag, train_bag=train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag]
    #Use laplace smoothing to prevent zero probability
    tag_count = len(taglist) + unique_tags
    w_in_tag = [pair[0] for pair in taglist if pair[0] == word]
    word_count_given_tag = len(w_in_tag) + 1
    return (word_count_given_tag, tag_count)

#### Function to compute transition probabilties for a given tag and previous tag

In [9]:
#Compute transition probabilities of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    t1_tags = [tag for tag in tags if tag==t1]  
    count_of_t1 = len(t1_tags) + unique_tags 
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2] 
    count_t2_given_t1 = len(t2_given_t1) + 1  
    return(count_t2_given_t1,count_of_t1)

In [10]:
t2_given_t1('NOUN','ADJ')

(122, 201)

In [11]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [12]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [13]:
tags_df

,SCONJ,NUM,X,DET,PROPN,ADJ,VERB,ADP,AUX,NOUN,CCONJ,PRON,SYM,PUNCT,PART,ADV
SCONJ,0.067568,0.013514,0.013514,0.202703,0.027027,0.067568,0.054054,0.040541,0.040541,0.121622,0.013514,0.243243,0.013514,0.054054,0.013514,0.013514
NUM,0.017857,0.017857,0.017857,0.017857,0.035714,0.071429,0.017857,0.053571,0.017857,0.142857,0.035714,0.017857,0.017857,0.482143,0.017857,0.017857
X,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.100000,0.050000,0.050000,0.050000,0.200000,0.050000,0.050000
DET,0.003356,0.006711,0.003356,0.016779,0.077181,0.218121,0.046980,0.010067,0.020134,0.489933,0.003356,0.026846,0.003356,0.003356,0.003356,0.067114
PROPN,0.005917,0.100592,0.005917,0.023669,0.183432,0.023669,0.023669,0.065089,0.017751,0.124260,0.076923,0.005917,0.023669,0.260355,0.053254,0.005917
ADJ,0.024876,0.024876,0.004975,0.009950,0.019900,0.039801,0.014925,0.089552,0.014925,0.606965,0.024876,0.004975,0.004975,0.099502,0.004975,0.009950
VERB,0.047761,0.005970,0.002985,0.170149,0.011940,0.062687,0.029851,0.241791,0.005970,0.140299,0.005970,0.104478,0.002985,0.071642,0.002985,0.092537
ADP,0.006579,0.029605,0.006579,0.407895,0.088816,0.078947,0.121711,0.016447,0.016447,0.118421,0.003289,0.049342,0.003289,0.016447,0.003289,0.032895
AUX,0.007752,0.023256,0.007752,0.100775,0.015504,0.139535,0.310078,0.031008,0.046512,0.031008,0.015504,0.069767,0.007752,0.038760,0.054264,0.100775
NOUN,0.014210,0.005329,0.001776,0.017762,0.017762,0.003552,0.097691,0.202487,0.063943,0.166963,0.046181,0.030195,0.001776,0.277087,0.021314,0.031972


#### Viterbi Algorithm

The steps are as follows:

1. Given a sequence of words
2. iterate through the sequence
3. for each word (starting from first word in sequence) calculate the product of emission probabilties and transition probabilties for all possible tags.
4. assign the tag which has maximum probability obtained in step 3 above.
5. move to the next word in sequence to repeat steps 3 and 4 above.

In [14]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['PUNCT', tag]
#                 continue
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Testing Viterbi Algorithm on test data

In [32]:
random.seed(19522298)

# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
tagged_seq = Viterbi(test_tagged_words)

In [33]:
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
vanilla_viterbi_accuracy = len(check)/len(tagged_seq)
print("The accuracy of the Vanilla Viterbi Algorithm is -", str(vanilla_viterbi_accuracy*100) + '%')

The accuracy of the Vanilla Viterbi Algorithm is - 61.81980374665478%
